# 📋 Ejercicio 3.1: Plan-Execute-Evaluate

**Módulo 3: Agentes Autónomos**  
**Tiempo estimado**: 17 minutos  
**Ejercicio**: 3.1

---

## 🎯 Objetivos de Aprendizaje

1. ✅ Implementar planner que crea plan completo
2. ✅ Executor que ejecuta paso a paso
3. ✅ Evaluator que verifica progreso
4. ✅ Loop de ejecución con límites


In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📋 SETUP Y VERIFICACIÓN                                  ║
# ╚════════════════════════════════════════════════════════════╝

%pip install -q langgraph langchain-openai python-dotenv

import os, sys
from dotenv import load_dotenv
sys.path.append(os.path.abspath('../..'))
load_dotenv()

print('='*50)
print('   SETUP VERIFICATION')
print('='*50)
print(f"✅ Python {sys.version.split()[0]}")
print(f"{'✅' if os.getenv('OPENAI_API_KEY') else '❌'} OpenAI API Key")
print(f"\n🎬 Ready!\n")

## 💬 SCRIPT

> "Plan-Execute: El agente PLANIFICA primero, EJECUTA después.
> Como un chef que lee toda la receta antes de cocinar."

In [ ]:
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

class PlanExecuteState(TypedDict):
    query: str
    plan: List[str]
    current_step: int
    results: List[str]
    evaluation: str

llm = ChatOpenAI(model='gpt-4o-mini')
print('✅ Setup')

In [ ]:
# Planner

def planner(state):
    '''Crea plan completo'''
    query = state['query']
    
    prompt = f'''Crea un plan de 3 pasos para: {query}
    
    Formato:
    1. [paso 1]
    2. [paso 2]
    3. [paso 3]'''
    
    plan_text = llm.invoke(prompt).content
    steps = [line.strip() for line in plan_text.split('\n') if line.strip() and line[0].isdigit()]
    
    print(f'📋 Plan creado con {len(steps)} pasos')
    for step in steps:
        print(f'  {step}')
    
    return {'plan': steps, 'current_step': 0, 'results': []}

print('✅ Planner definido')

In [ ]:
# Executor

def executor(state):
    '''Ejecuta paso actual'''
    plan = state['plan']
    current = state['current_step']
    
    if current >= len(plan):
        return state
    
    step = plan[current]
    print(f'⚙️ Ejecutando paso {current+1}: {step[:50]}...')
    
    # Simular ejecución
    result = f'Resultado del paso {current+1}'
    
    return {
        'results': state['results'] + [result],
        'current_step': current + 1
    }

print('✅ Executor definido')

In [ ]:
# Decisión de continuar

def should_continue(state):
    current = state['current_step']
    total = len(state['plan'])
    
    # Límite de seguridad
    if current > 10:
        return 'end'
    
    if current >= total:
        return 'evaluate'
    return 'execute'

print('✅ Router definido')

In [ ]:
# Evaluator

def evaluator(state):
    '''Evalúa si el plan funcionó'''
    print('✅ Evaluando resultados...')
    eval = f'Plan completado. {len(state["results"])} pasos ejecutados.'
    return {'evaluation': eval}

print('✅ Evaluator definido')

In [ ]:
# Construir grafo

graph = StateGraph(PlanExecuteState)

graph.add_node('planner', planner)
graph.add_node('executor', executor)
graph.add_node('evaluator', evaluator)

graph.add_edge(START, 'planner')
graph.add_edge('planner', 'executor')
graph.add_conditional_edges(
    'executor',
    should_continue,
    {
        'execute': 'executor',  # Loop
        'evaluate': 'evaluator',
        'end': END
    }
)
graph.add_edge('evaluator', END)

app = graph.compile()
print('✅ Plan-Execute compilado')

In [ ]:
from IPython.display import Image, display
# View
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# 🎬 DEMO

query = 'Investiga las tendencias de IA en 2024 y crea un reporte'

print('='*60)
print('📋 PLAN-EXECUTE-EVALUATE')
print('='*60)
print(f'Query: {query}\n')

result = app.invoke({'query': query})

print(f'\nPlan: {result["plan"]}')
print(f'\nResultados: {result["results"]}')
print(f'\nEvaluación: {result["evaluation"]}')

---

## ✅ CHECKPOINT FINAL

- [ ] Código ejecuta sin errores
- [ ] Conceptos clave entendidos
- [ ] Listos para continuar

### 💬 PREGUNTA:
> "¿Alguna duda antes de continuar?"
